In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score, mean_squared_error, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [ ]:
path = '/content/drive/My Drive/Kaggle_Competitions/titanic/test.csv'
test = pd.read_csv(path)
path2 = '/content/drive/My Drive/Kaggle_Competitions/titanic/train.csv'
train = pd.read_csv(path2)

In [ ]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
train.drop(columns = ['Cabin', 'PassengerId', 'Name', 'Ticket'], inplace = True)

In [ ]:
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode()[0])
test2 = test.copy()
test2.drop(columns = ['Cabin','Name','Ticket','PassengerId'], inplace = True)

In [ ]:
test2

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.50000,0,0,7.8292,Q
1,3,female,47.00000,1,0,7.0000,S
2,2,male,62.00000,0,0,9.6875,Q
3,3,male,27.00000,0,0,8.6625,S
4,3,female,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,8.0500,S
414,1,female,39.00000,0,0,108.9000,C
415,3,male,38.50000,0,0,7.2500,S
416,3,male,30.27259,0,0,8.0500,S


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [ ]:
one_hot_encoder = OneHotEncoder(sparse_output = False)
categorical_columns = ['Sex', 'Embarked']
df_encoded = one_hot_encoder.fit_transform(train[categorical_columns])

feature_names = one_hot_encoder.get_feature_names_out(categorical_columns)

new_df = pd.DataFrame(df_encoded, columns = feature_names)

df2 = pd.concat([train.drop(columns = categorical_columns), new_df], axis = 1)

In [ ]:
one_hot_encoder = OneHotEncoder(sparse_output = False)
categorical_columns = ['Sex', 'Embarked']
df2_encoded = one_hot_encoder.fit_transform(test2[categorical_columns])

feature_names = one_hot_encoder.get_feature_names_out(categorical_columns)

new_df2 = pd.DataFrame(df2_encoded, columns = feature_names)

df_test2 = pd.concat([test2.drop(columns = categorical_columns), new_df2], axis = 1)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Sex_female  891 non-null    float64
 7   Sex_male    891 non-null    float64
 8   Embarked_C  891 non-null    float64
 9   Embarked_Q  891 non-null    float64
 10  Embarked_S  891 non-null    float64
dtypes: float64(7), int64(4)
memory usage: 76.7 KB


In [ ]:
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df2), columns = df2.columns)

In [ ]:
scaler = StandardScaler()
scaled_test = pd.DataFrame(scaler.fit_transform(df_test2), columns = df_test2.columns)

In [ ]:
scaled_test2 = pd.DataFrame(scaled_test, )
scaled_test2['PassengerId'] = test['PassengerId']
scaled_test2

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,PassengerId
0,0.873482,0.334993,-0.499470,-0.400248,-0.498407,-0.755929,0.755929,-0.568142,2.843757,-1.350676,892
1,0.873482,1.325530,0.616992,-0.400248,-0.513274,1.322876,-1.322876,-0.568142,-0.351647,0.740370,893
2,-0.315819,2.514175,-0.499470,-0.400248,-0.465088,-0.755929,0.755929,-0.568142,2.843757,-1.350676,894
3,0.873482,-0.259330,-0.499470,-0.400248,-0.483466,-0.755929,0.755929,-0.568142,-0.351647,0.740370,895
4,0.873482,-0.655545,0.616992,0.619896,-0.418471,1.322876,-1.322876,-0.568142,-0.351647,0.740370,896
...,...,...,...,...,...,...,...,...,...,...,...
413,0.873482,0.000000,-0.499470,-0.400248,-0.494448,-0.755929,0.755929,-0.568142,-0.351647,0.740370,1305
414,-1.505120,0.691586,-0.499470,-0.400248,1.313753,1.322876,-1.322876,1.760125,-0.351647,-1.350676,1306
415,0.873482,0.651965,-0.499470,-0.400248,-0.508792,-0.755929,0.755929,-0.568142,-0.351647,0.740370,1307
416,0.873482,0.000000,-0.499470,-0.400248,-0.494448,-0.755929,0.755929,-0.568142,-0.351647,0.740370,1308


In [ ]:
X = scaled_df.drop(columns = ['Survived'], axis = 1)
y = df2['Survived']

In [ ]:
X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,-0.737695,0.737695,-0.482043,-0.307562,0.615838
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,1.355574,-1.355574,2.074505,-0.307562,-1.623803
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,1.355574,-1.355574,-0.482043,-0.307562,0.615838
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,1.355574,-1.355574,-0.482043,-0.307562,0.615838
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,-0.737695,0.737695,-0.482043,-0.307562,0.615838
...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,-0.207709,-0.474545,-0.473674,-0.386671,-0.737695,0.737695,-0.482043,-0.307562,0.615838
887,-1.566107,-0.823344,-0.474545,-0.473674,-0.044381,1.355574,-1.355574,-0.482043,-0.307562,0.615838
888,0.827377,0.000000,0.432793,2.008933,-0.176263,1.355574,-1.355574,-0.482043,-0.307562,0.615838
889,-1.566107,-0.284663,-0.474545,-0.473674,-0.044381,-0.737695,0.737695,2.074505,-0.307562,-1.623803


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression()

In [ ]:
model = RandomForestClassifier(n_estimators = 200, max_depth = 2)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, n_estimators=200)

In [ ]:
model3 = AdaBoostClassifier(n_estimators = 200)
model3.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=200)

In [ ]:
#set ids as PassengerId and predict survival
ids = scaled_test2['PassengerId']
predictions = model3.predict(scaled_test2.drop('PassengerId', axis=1))

#set the output as a dataframe and convert to csv file named submission.csv
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('submission.csv', index=False)